# Kaggle: Kobe Bryant Shot Selection (Model)

This data contains the location and circumstances of every field goal attempted by Kobe Bryant took during his 20-year career. Your task is to predict whether the basket went in (shot_made_flag).

We have removed 5000 of the shot_made_flags (represented as missing values in the csv file). These are the test set shots for which you must submit a prediction. You are provided a sample submission file with the correct shot_ids needed for a valid prediction.

To avoid leakage, your method should only train on events that occurred prior to the shot for which you are predicting! Since this is a playground competition with public answers, it's up to you to abide by this rule.

https://www.kaggle.com/c/kobe-bryant-shot-selection/data

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import style; style.use('ggplot')

In [2]:
# load data (already sorted by shot_id, order is necessary for submission)
data = pd.read_csv('data.csv')
data.head()

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
0,Jump Shot,Jump Shot,10,20000012,33.9723,167,72,-118.1028,10,1,...,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,1
1,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1,...,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,2
2,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,7,1,...,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,3
3,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,6,1,...,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,4
4,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,6,2,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,5


## Plan:
In visualization notebook:
- separate train and test data
    - use all labeled data for train, all unlabeled for test
    - remember to perform all preprocessing on BOTH train and test data together
- explore train data
    - determine trends
    - make visualizations

In model notebook:
- determine relevant features
- preprocess relevant features
    - BOTH train and test together
- determine model type
    - test various models using cross validation
        - break train set into train and validation sets
    - record best model
- make predictions on test set using best model

In [3]:
list(data)

['action_type',
 'combined_shot_type',
 'game_event_id',
 'game_id',
 'lat',
 'loc_x',
 'loc_y',
 'lon',
 'minutes_remaining',
 'period',
 'playoffs',
 'season',
 'seconds_remaining',
 'shot_distance',
 'shot_made_flag',
 'shot_type',
 'shot_zone_area',
 'shot_zone_basic',
 'shot_zone_range',
 'team_id',
 'team_name',
 'game_date',
 'matchup',
 'opponent',
 'shot_id']

## Features to save:
- action_type/combined_shot_type: type of shot
    - yes, only use one
    - one-hot encode from string
- game_event_id/game_id: specific game when shot was taken
    - maybe, only use game_id
        - 1558 values: corresponds to total number of games Kobe played
    - one hot encode from int
- lat/lon: lattitude and longitude
    - maybe -- is this where in the country the game occured?
- loc_x/loc_y: x and y coordinates on court where shot was taken
    - yes
- minutes_remaining
    - yes
    - no preprocessing?
- period: quarter (or overtime, up to triple overtime (7))
    - yes
    - one hot encode from int
- playoffs
    - yes
    - already one hot encoded
- season: date range in years
    - yes
    - one-hot encode from string
- seconds_remaining
    - maybe -- range from 0-59, what is this referring to?
- shot_distance: from basket
    - yes
    - no preprocessing
- shot_type: 2pt or 3pt
    - yes
    - one hot encode
- shot_zone_area: left, right, center
    - yes
    - one hot encode
- shot_zone_basic: paint, mid-range, etc.
    - yes
    - one hot encode
- shot_zone_range
    - maybe (watch for double counting with distance)
    - one hot encode
- team_id/team_name: lakers (single value)
    - no
- game_date: year, month, day
    - no
- matchup/opponent
    - yes
    - matchup: opponent AND home (vs) or away (@)
    - opponent: just opponent
- shot_id: unique value for every shot
    - no

## Label:
- shot_made_flag is the label column

## Preprocessing
Remember to apply preprocessing on ALL of the data.

In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_encoder = LabelEncoder()
one_hot_encoder = OneHotEncoder()

In [5]:
# combined_shot_type: one-hot encode from string
combined_shot_type = one_hot_encoder.fit_transform(data.combined_shot_type.values.reshape(-1, 1)).toarray()

In [6]:
# period: one-hot encode from int
period = one_hot_encoder.fit_transform(data.period.values.reshape(-1, 1)).toarray()

In [7]:
# season: one-hot encode from string
season = one_hot_encoder.fit_transform(data.season.values.reshape(-1, 1)).toarray()

In [8]:
# shot_type: one-hot encode from string
shot_type = one_hot_encoder.fit_transform(data.shot_type.values.reshape(-1, 1)).toarray()

In [9]:
# shot_zone_area: one-hot encode from string
shot_zone_area = one_hot_encoder.fit_transform(data.shot_zone_area.values.reshape(-1, 1)).toarray()

In [10]:
# shot_zone_basic: one-hot encode from string
shot_zone_basic = one_hot_encoder.fit_transform(data.shot_zone_basic.values.reshape(-1, 1)).toarray()

In [11]:
# put all relevant features together
X = np.hstack((combined_shot_type,
               period,
               data.playoffs.values.reshape(-1, 1),
               season,
               data.shot_distance.values.reshape(-1, 1),
               shot_type,
               shot_zone_area,
               shot_zone_basic))

# store labels
y = data[np.isnan(data.shot_made_flag) == False].shot_made_flag.values

In [12]:
X.shape, y.shape

((30697, 50), (25697,))

## Train/Validation/Test Split

In [13]:
# train/test split
X_train = X[np.isnan(data.shot_made_flag) == False]
y_train = y
X_test = X[np.isnan(data.shot_made_flag) == True]

In [14]:
# save train set before splitting into train and validation (to train final model)
old_X_train = X_train
old_y_train = y_train

# further split train into train/validation
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, 
                                                    test_size=0.2, 
                                                    shuffle=True)

In [15]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((20557, 50), (5140, 50), (20557,), (5140,))

## Models

In [16]:
# binary accuracy if predicting every shot as a miss
from sklearn.metrics import accuracy_score
accuracy_score(y_val, np.zeros(y_val.shape))

0.5459143968871595

In [17]:
""" DONT RUN """
# random forest
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=100)
forest.fit(X_train, y_train)
accuracy_score(y_val, forest.predict(X_val))

In [ ]:
""" DONT RUN """
# neural net
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(hidden_layer_sizes=(500, 250, 125, 50, 25, 10, 5), max_iter=1000)
nn.fit(X_train, y_train)
accuracy_score(y_val, nn.predict(X_val))

In [ ]:
""" DONT RUN """
# svm
from sklearn.svm import SVC
svm = SVC(kernel='rbf', gamma='auto', probability=True)
svm.fit(X_train, y_train)
accuracy_score(y_val, svm.predict(X_val))

In [ ]:
""" DONT RUN """
# logloss for svm
from sklearn.metrics import log_loss
y_proba = svm.predict_proba(X_val)
log_loss(y_val, y_proba)

In [23]:
# train svm on all data, then make predictions
from sklearn.svm import SVC
svm = SVC(kernel='rbf', gamma='auto', probability=True)
svm.fit(old_X_train, old_y_train)
predictions = svm.predict_proba(X_test)

In [26]:
predictions[:10, :]

array([[0.60979239, 0.39020761],
       [0.60172834, 0.39827166],
       [0.35795181, 0.64204819],
       [0.35791859, 0.64208141],
       [0.60981135, 0.39018865],
       [0.60957634, 0.39042366],
       [0.35959864, 0.64040136],
       [0.35959864, 0.64040136],
       [0.35795181, 0.64204819],
       [0.60979277, 0.39020723]])

In [27]:
test_df = data[np.isnan(data.shot_made_flag)]
test_df

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
0,Jump Shot,Jump Shot,10,20000012,33.9723,167,72,-118.1028,10,1,...,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,1
7,Jump Shot,Jump Shot,254,20000012,34.0163,1,28,-118.2688,8,3,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,8
16,Driving Layup Shot,Layup,100,20000019,34.0443,0,0,-118.2698,0,1,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-11-01,LAL vs. UTA,UTA,17
19,Driving Layup Shot,Layup,249,20000019,34.0443,0,0,-118.2698,10,3,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-11-01,LAL vs. UTA,UTA,20
32,Jump Shot,Jump Shot,4,20000047,33.9683,163,76,-118.1068,11,1,...,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-11-04,LAL @ VAN,VAN,33
33,Jump Shot,Jump Shot,8,20000047,33.8503,70,194,-118.1998,10,1,...,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-11-04,LAL @ VAN,VAN,34
34,Layup Shot,Layup,26,20000047,34.0253,1,19,-118.2688,7,1,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-11-04,LAL @ VAN,VAN,35
35,Layup Shot,Layup,37,20000047,34.0293,-12,15,-118.2818,5,1,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-11-04,LAL @ VAN,VAN,36
36,Reverse Layup Shot,Layup,53,20000047,34.0403,1,4,-118.2688,4,1,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-11-04,LAL @ VAN,VAN,37
37,Jump Shot,Jump Shot,165,20000047,33.9283,-117,116,-118.3868,5,2,...,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-11-04,LAL @ VAN,VAN,38


Based on above two cells, predictions in form [P(miss), P(make)], same as [P(0), P(1)]

Only save P(make)?

In [73]:
# setup np array for submission
submission = np.hstack((test_df.shot_id.values.reshape(-1, 1).astype(object), 
                        predictions[:, -1].reshape(-1, 1).astype(object)))

submission = np.vstack((np.array(['shot_id', 'shot_made_flag']),
                       submission))

In [74]:
submission

array([['shot_id', 'shot_made_flag'],
       [1, 0.39020760901609397],
       [8, 0.3982716642249478],
       ...,
       [30683, 0.3893635898660541],
       [30687, 0.3919009177761938],
       [30694, 0.39671233492456703]], dtype=object)

In [75]:
# write predictions and indices to csv
np.savetxt('kobe_predictions.csv', 
           submission, 
           delimiter=',', 
           fmt='%s')